In [ ]:
# 📦 Install necessary packages
!pip install requests nltk
!pip install textblob
!pip install random
!pip install time
# 📚 Import libraries
import requests
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# 🧠 Predefined crypto dataset for rule-based logic
crypto_db = {
    "Bitcoin": {
        "price_trend": "rising",
        "market_cap": "high",
        "energy_use": "high",
        "sustainability_score": 3/10
    },
    "Ethereum": {
        "price_trend": "stable",
        "market_cap": "high",
        "energy_use": "medium",
        "sustainability_score": 6/10
    },
    "Cardano": {
        "price_trend": "rising",
        "market_cap": "medium",
        "energy_use": "low",
        "sustainability_score": 8/10
    }
}
# 🌐 Real-time API function
def get_price_from_api(coin_id):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        price = data['market_data']['current_price']['usd']
        return price
    else:
        return None
